In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install keras-self-attention

In [ ]:
!pip install emoji

In [ ]:
!pip install ekphrasis

In [ ]:
!pip install transformers==4.2.1

In [ ]:
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os
from collections import Counter

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [ ]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

In [ ]:
def print_text(texts,i,j):
    for u in range(i,j):
        print(texts[u])
        print()

In [ ]:
df_1 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2016train-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_1.head(5)) #last N rows
# print(len(df_1))

df_2 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2016test-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_2.head(5)) #last N rows
# print(len(df_2))

df_3 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2016devtest-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_3.head(5)) #last N rows
# print(len(df_3))

df_4 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2016dev-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_4.head(5)) #last N rows
# print(len(df_4))

df_5 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2015train-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_5.head(5)) #last N rows
# print(len(df_5))

df_6 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2015test-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_6.head(5)) #last N rows
# print(len(df_6))

df_7 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2014test-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_7.head(5)) #last N rows
# print(len(df_7))

df_8 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2014sarcasm-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_8.head(5)) #last N rows
# print(len(df_8))

df_9 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2013train-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_9.head(5)) #last N rows
# print(len(df_9))

df_10 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2013test-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_10.head(5)) #last N rows
# print(len(df_10))

df_11 = pd.read_csv('/content/drive/My Drive/Semeval 2017/twitter-2013dev-A.txt', delimiter='\t', encoding='utf-8', header=None)
# print(df_11.head(5)) #last N rows
# print(len(df_11))


<h2>Balancing the data</h2>

In [ ]:
df = pd.DataFrame()
df = df.append(df_1, ignore_index = True)
df = df.append(df_2, ignore_index = True)
df = df.append(df_3, ignore_index = True)
df = df.append(df_4, ignore_index = True)

df = df.append(df_5, ignore_index = True)
df = df.append(df_6, ignore_index = True)
df = df.append(df_7, ignore_index = True)
df = df.append(df_8, ignore_index = True)

df = df.append(df_9, ignore_index = True)
df = df.append(df_10, ignore_index = True)
df = df.append(df_11, ignore_index = True)

print(df.head(5))
print(len(df))

In [ ]:
# Testing for null values
# lol = np.asarray(df_[1].isnull())

# for i in range(0,len(lol)):
#     if lol[i]:
#         print(i)

In [ ]:
print(len(df))

In [ ]:
text_array = df[2]
labels = df[1]
print("Length of training data: ",len(text_array))
print_text(text_array,0,10)

In [ ]:
df_val = pd.read_csv('/content/drive/My Drive/Semeval 2017/Test/SemEval2017-task4-test.subtask-A.english.txt', delimiter='\n', encoding='utf-8', header=None)
print(df_val.tail(5)) #last N rows
print(len(df_val))

In [ ]:
lol = []
test_set = np.asarray(df_val[0])
for i in range(0,len(df_val)):
    temp = np.asarray(test_set[i].split("\t"))
    temp = temp.reshape((3))
    lol.append(temp)

In [ ]:
df_val = pd.DataFrame(lol)
df_val.head(5)

In [ ]:
text_array_val = df_val[2]
labels_val = df_val[1]
print("Length of validation data: ",len(text_array_val))
print_text(text_array_val,0,10)

In [ ]:
print(Counter(labels))
print(Counter(labels_val))

In [ ]:
#removing website names
def remove_website(text):
    return " ".join([word if re.search("r'https?://\S+|www\.\S+'|((?i).com$|.co|.net)",word)==None else "" for word in text.split(" ") ])

# Training set 
text_array = text_array.apply(lambda text: remove_website(text))
print_text(text_array,0,10)

print("**************************************************************************")

# Validation set 
text_array_val = text_array_val.apply(lambda text: remove_website(text))
print_text(text_array_val,0,10)

In [ ]:
# Functions for chat word conversion
f = open("/content/drive/My Drive/Semeval 2017/slang.txt", "r")
chat_words_str = f.read()
chat_words_map_dict = {}
chat_words_list = []

for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [ ]:
# Chat word conversion
# Training set
text_array = text_array.apply(lambda text: chat_words_conversion(text))
print_text(text_array,0,10)

print("********************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: chat_words_conversion(text))
print_text(text_array_val,0,10)

In [ ]:
os.chdir("/content/drive/My Drive/Semeval 2017")

In [ ]:
#Function for emoticon conversion
from emoticons import EMOTICONS

def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', " ".join(EMOTICONS[emot].replace(",","").split()), text)
    return text


#testing the emoticon function
text = "Hello :-) :-)"
text = convert_emoticons(text)
print(text + "\n")

In [ ]:
# Emoticon conversion
# Training set
text_array = text_array.apply(lambda text: convert_emoticons(text))
print_text(text_array,0,10)

print("**********************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: convert_emoticons(text))
print_text(text_array_val,0,10)

In [ ]:
os.chdir("/content")

In [ ]:
# FUnction for removal of emoji
import emoji

def convert_emojis(text):
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = re.sub("_|-"," ",text)
    return text

# Training set
text_array = text_array.apply(lambda text: convert_emojis(text))
print_text(text_array,0,10)

print("**************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: convert_emojis(text))
print_text(text_array_val,0,10)

In [ ]:
# Ekphrasis pipe for text pre-processing
def ekphrasis_pipe(sentence):
    cleaned_sentence = " ".join(text_processor.pre_process_doc(sentence))
    return cleaned_sentence

# Training set
text_array = text_array.apply(lambda text: ekphrasis_pipe(text))
print("Training set completed.......")
#Validation set
text_array_val = text_array_val.apply(lambda text: ekphrasis_pipe(text))
print("Test set completed.......")

In [ ]:
print_text(text_array,0,10)
print("************************************************************************")
print_text(text_array_val,0,10)

In [ ]:
# Removing unnecessary punctuations
PUNCT_TO_REMOVE = "\"$%&'()+,-./;=[\]^_`{|}~"
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

# Training set
text_array = text_array.apply(lambda text: remove_punctuation(text))
print_text(text_array,0,10)

print("********************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: remove_punctuation(text))
print_text(text_array_val,0,10)

In [ ]:
# Finding length of longest array
maxLen = len(max(text_array,key = lambda text: len(text.split(" "))).split(" "))
print(maxLen)

In [ ]:
u = lambda text: len(text.split(" "))
sentence_lengths = []
for x in text_array:
    sentence_lengths.append(u(x))
print(sorted(sentence_lengths)[-800:])
print(len(sentence_lengths))

In [ ]:
# Count of each label in dataset
from collections import Counter

# Printing training set counts for analysis
print("Elements: ",set(labels))
print("Length: ",len(labels))
print(Counter(labels))

print("**************************************************************************")

# Printing validation set counts for analysis
print("Elements: ",set(labels_val))
print("Length: ",len(labels_val))
print(Counter(labels_val))

In [ ]:
Y = []
Y_val = []

# Training set    
for i in range(0,len(labels)):
    if(labels[i] == 'neutral'):
        Y.append(0)
    if(labels[i] == 'positive'):
        Y.append(1)
    if(labels[i] == 'negative'):
        Y.append(2)


# Validation set
for i in range(0,len(labels_val)):
    if(labels_val[i] == 'neutral'):
        Y_val.append(0)
    if(labels_val[i] == 'positive'):
        Y_val.append(1)
    if(labels_val[i] == 'negative'):
        Y_val.append(2)

In [ ]:
print(len(Y),len(Y_val))

In [ ]:
print(Counter(Y))
print(Counter(Y_val))

In [ ]:
# Testing the conversion into integers
for i in range(310,320):
    print(text_array_val[i])
    print(labels_val[i],Y_val[i])

In [ ]:
# Verifying train set 
X = np.asarray(list(text_array))
Y = np.asarray(list(Y))
labels = np.asarray(list(labels))
print(type(X))
print(type(Y))
print(type(labels))
print(np.shape(X),np.shape(Y),np.shape(labels))

# Verifying validation set
X_val = np.asarray(list(text_array_val))
Y_val = np.asarray(list(Y_val))
labels_val = np.asarray(list(labels_val))
print(type(X_val))
print(type(Y_val))
print(type(labels_val))
print(np.shape(X_val),np.shape(Y_val),np.shape(labels_val))

In [ ]:
index = 824
print(X[index])
print(labels[index])
print(Y[index])

In [ ]:
print(type(X))
print(type(Y))
print(np.shape(X),np.shape(Y),np.shape(labels))
print(np.shape(X_val),np.shape(Y_val),np.shape(labels_val))

In [ ]:
# Converting to one hot vectors
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)] #u[Y] helps to index each element of Y index at u. U here is a class array
    return Y

In [ ]:
Y_oh_train = convert_to_one_hot(np.array(Y), C = 3)
Y_oh_val = convert_to_one_hot(np.array(Y_val), C = 3)

print(np.shape(Y_oh_train))
index = 310
print(labels[index], Y[index], "is converted into one hot", Y_oh_train[index])



<h2>Tensorflow Model</h2>

In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os
from collections import Counter

In [ ]:
from transformers import RobertaTokenizerFast, TFRobertaModel, TFBertModel, BertTokenizerFast, ElectraTokenizerFast, TFElectraModel, AlbertTokenizerFast, TFAlbertModel, XLNetTokenizerFast, TFXLNetModel, MPNetTokenizerFast, TFMPNetModel
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from keras_self_attention import SeqSelfAttention

In [ ]:
print(tf.__version__)

In [ ]:

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
tokenizer = MPNetTokenizerFast.from_pretrained("microsoft/mpnet-base")

In [ ]:
X = list(X)
X_val = list(X_val)

In [ ]:
train_encodings = tokenizer(X, max_length=80, truncation=True, padding="max_length", return_tensors='tf')
val_encodings = tokenizer(X_val, max_length=80, truncation=True, padding="max_length", return_tensors='tf')

In [ ]:
print(np.shape(train_encodings["input_ids"]))
print(np.shape(val_encodings["input_ids"]))

In [ ]:
print(train_encodings["input_ids"][0])
print("***************************************************************************")
print(val_encodings["input_ids"][0])

In [ ]:
# This is the best model
def Offense_classifier(input_shape):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """

    model = TFMPNetModel.from_pretrained('microsoft/mpnet-base')
    layer = model.layers[0]

    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    inputs = keras.Input(shape=input_shape, dtype='int32')
    input_masks = keras.Input(shape=input_shape, dtype='int32')
    
    embeddings = layer([inputs, input_masks])[0][:,0,:]
    
    # embeddings = keras.layers.GaussianNoise(0.2)(embeddings)

    # embeddings = keras.layers.Dropout(0.3)(embeddings)

    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    # lstm_one = keras.layers.Bidirectional(keras.layers.LSTM(150, return_sequences=True, recurrent_dropout=0.25, dropout=0.2)) 
    # X = lstm_one(embeddings)
    # X = keras.layers.Dropout(0.2)(X)

    # lstm_two = keras.layers.Bidirectional(keras.layers.LSTM(150, return_sequences=True, recurrent_dropout=0.25, dropout=0.2)) 
    # X = lstm_two(X)
    # X = keras.layers.Dropout(0.2)(X)

    # # *************Attention*******************
    # X = SeqSelfAttention(attention_activation='elu')(X)
    # # ****************Attention*******************

    # post_activation_GRU_cell = keras.layers.GRU(64, return_sequences = False, recurrent_dropout=0.25, dropout=0.2)
    # X = post_activation_GRU_cell(X)

    X = keras.layers.Dense(32,activation='elu',kernel_regularizer=keras.regularizers.l2(0.0001))(embeddings)

    X = keras.layers.BatchNormalization(momentum=0.99, epsilon=0.001, center=True, scale=True)(X)

    X = keras.layers.Dense(3,activation='tanh',kernel_regularizer=keras.regularizers.l2(0.0001))(X)

    
    # Add a sigmoid activation
    X = keras.layers.Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = keras.Model(inputs=[inputs,input_masks], outputs=[X])
    
    
    return model

In [ ]:
model = Offense_classifier((80,))
model.summary()

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
class EvaluationMetric(keras.callbacks.Callback):   
    
    def __init__(self, trial_encodings, trial_masks, Y_val):
        super(EvaluationMetric, self).__init__()
        self.trial_encodings = trial_encodings
        self.trial_masks = trial_masks
        self.Y_val = Y_val
    
    def on_epoch_begin(self, epoch, logs={}):
        print("\nTraining...")

    def on_epoch_end(self, epoch, logs={}):
        print("\nEvaluating...")
        trial_prediction = self.model.predict([self.trial_encodings,self.trial_masks])
        
        pred = []
        for i in range(0,len(self.Y_val)):
            num = np.argmax(trial_prediction[i])
            pred.append(num)
        
        from sklearn.metrics import classification_report
        print(classification_report(Y_val, pred, digits=3))
        
evaluation_metric = EvaluationMetric(val_encodings["input_ids"], val_encodings["attention_mask"], Y_val)

In [ ]:
with strategy.scope():
    model = Offense_classifier((80,))
    optimizer = keras.optimizers.Adam(learning_rate=1e-5)
    loss_fun = [
          tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    ]
    metric = ['acc']
    model.compile(optimizer=optimizer, loss=loss_fun, metrics=metric)

In [ ]:
model.summary()

In [ ]:
checkpoint = ModelCheckpoint(filepath='/content/neutro-mpnet.{epoch:03d}.h5',
                                 verbose = 0,
                                 save_weights_only=True,
                                 epoch=4)

In [ ]:
c = Counter(Y)
print(c)
print(c.keys())
neutral = c[0]
pos = c[1]
neg = c[2]
total = pos+neg+neutral
print(neutral,pos,neg,total)

In [ ]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
maxi = max(pos,neg,neutral)
weight_for_0 = (maxi / (maxi+neutral))
weight_for_1 = (maxi / (maxi+pos))
weight_for_2 = (maxi / (maxi+neg))

class_weight_ = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))

In [ ]:
history = model.fit(
    x = [train_encodings["input_ids"], train_encodings["attention_mask"]],
    y = Y_oh_train,
    validation_data = ([val_encodings["input_ids"],val_encodings["attention_mask"]],Y_oh_val),
    callbacks = [evaluation_metric, checkpoint],
    batch_size = 32,
    shuffle=True,
    epochs=6,
    class_weight = class_weight_
)

In [ ]:
# plot_model(model, to_file="model.png", show_shapes=True, show_layer_names=False)

In [ ]:
model.load_weights("/content/drive/MyDrive/semeval 17 transformer weights/neutro-mpnet.004.h5")
# model.save_weights("/content/drive/MyDrive/semeval 17 transformer weights/neutro-mpnet.004.h5")

In [ ]:
answer = model.predict([val_encodings["input_ids"],val_encodings["attention_mask"]])

In [ ]:
print(X_val[0])
print(Y_oh_val[0])
print(labels_val[0])
print("******************************************")
print(len(answer),len(answer))

In [ ]:
Counter(Y_val)

In [ ]:
# used for querying
count_sl = 0
count_pos = 0
count_not = 0
pred = []
text = df_val[2]

temp = 0
for i in range(0,len(X_val)):
    num = np.argmax(answer[i])
    pred.append(num)

print(temp)

0


In [ ]:
Counter(pred)

In [ ]:
Counter(Y_val)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_val, predictions=pred, dtype=tf.dtypes.int32)
print(con_mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat, annot=True,cmap=plt.cm.Spectral,fmt='d',xticklabels=["Neutral","Positive","Negative"], yticklabels=["Neutral","Positive","Negative"])
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
from sklearn.metrics import f1_score
f1_score(Y_val, pred, average='macro')

In [ ]:
from sklearn.metrics import recall_score
recall_score(Y_val, pred, average='macro')

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Neutral', 'Positive', 'Negative']
print(classification_report(Y_val, pred, digits=3))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_val, pred, normalize=True)

<h3>Clustering</h3>

In [ ]:
pip install plotly==4.5.4

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
flag = []
count = 0

positive = []
negative = []
neutral = []


for i in range(0,len(pred)):
    count = count + 1
    neutral.append(answer[i][0])
    positive.append(answer[i][1])
    negative.append(answer[i][2])


print(count)

In [ ]:
pred_colour = []
for i in range(0,len(pred)):
    if pred[i] == 0:
      pred_colour.append("Neutral")
    if pred[i] == 1:
      pred_colour.append("Positive")
    if pred[i] == 2:
      pred_colour.append("Negative")

test_df = pd.DataFrame({'positive':positive, 'negative':negative, 'neutral':neutral, 'Prediction':pred_colour})

fig = px.scatter_3d(test_df, x='positive', y='negative', z='neutral', color='Prediction')
fig.update_traces(
    marker={
        'size': 0.7,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'})
fig.update_layout(width = 700)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

<h5>SVNS</h5>

<h3>Middle Layer</h3>

In [ ]:
model.layers[-3]

In [ ]:
with strategy.scope():
    cl_model = keras.Model(model.input, model.layers[-3].output)

In [ ]:
cl_32 = cl_model.predict([val_encodings["input_ids"],val_encodings["attention_mask"]])

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=4).fit(cl_32)
y_kmeans_batchnorm = kmeans.predict(cl_32)

In [ ]:
for i in range(0,len(y_kmeans_batchnorm)):
    if(y_kmeans_batchnorm[i] == 0):
      y_kmeans_batchnorm[i] = 1
    elif(y_kmeans_batchnorm[i] == 1):
      y_kmeans_batchnorm[i] = 2
    else:
      y_kmeans_batchnorm[i] = 0

In [ ]:
centers_batchnorm = kmeans.cluster_centers_

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_val, predictions=y_kmeans_batchnorm)
print(con_mat)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Neutral', 'Positive', 'Negative']
print(classification_report(Y_val, y_kmeans_batchnorm, digits=3, target_names=target_names))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

figure = plt.figure(figsize=(8, 8))
sns.heatmap(con_mat, annot=True,cmap=plt.cm.Spectral,fmt='d',xticklabels=["Neutral","Positive","Negative"], yticklabels=["Neutral","Positive","Negative"])
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
svns_neu_bn = []
for i in range(0,len(Y_val)):
    neu = cosine(cl_32[i], centers_batchnorm[2])/2
    svns_neu_bn.append(1-neu)
print(len(svns_neu_bn))

In [ ]:
svns_pos_bn = []
for i in range(0,len(Y_val)):
    pos = cosine(cl_32[i], centers_batchnorm[0])/2
    svns_pos_bn.append(1-pos)
print(len(svns_pos_bn))

In [ ]:
svns_neg_bn = []
for i in range(0,len(Y_val)):
    neg = cosine(cl_32[i], centers_batchnorm[1])/2
    svns_neg_bn.append(1-neg)
print(len(svns_neg_bn))

In [ ]:
pred_colour = []
for i in range(0,len(pred)):
    if y_kmeans_batchnorm[i] == 0:
      pred_colour.append("Neutral")
    if y_kmeans_batchnorm[i] == 1:
      pred_colour.append("Positive")
    if y_kmeans_batchnorm[i] == 2:
      pred_colour.append("Negative")

test_df = pd.DataFrame({'SVNS Positive':svns_pos_bn, 'SVNS Negative':svns_neg_bn, 'SVNS Neutral':svns_neu_bn, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Positive', y='SVNS Negative', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'})
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

<h3>GRU</h3>

In [ ]:
model.layers[-5]

In [ ]:
with strategy.scope():
    cl_model = keras.Model(model.input, (model.layers[-5].output))

In [ ]:
cl_32 = cl_model.predict([val_encodings["input_ids"],val_encodings["attention_mask"]])

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=4).fit(cl_32)
y_kmeans_gru = kmeans.predict(cl_32)

In [ ]:
for i in range(0,len(y_kmeans_gru)):
    if(y_kmeans_gru[i] == 0):
      y_kmeans_gru[i] = 1
    elif(y_kmeans_gru[i] == 1):
      y_kmeans_gru[i] = 2
    else:
      y_kmeans_gru[i] = 0

In [ ]:
centers_gru = kmeans.cluster_centers_

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_val, predictions=y_kmeans_gru)
print(con_mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

figure = plt.figure(figsize=(8, 8))
sns.set(font_scale=1.5)
sns.heatmap(con_mat, annot=True,cmap=plt.cm.Spectral,fmt='d',xticklabels=["Neutral","Positive","Negative"], yticklabels=["Neutral","Positive","Negative"])
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Neutral', 'Positive', 'Negative']
print(classification_report(Y_val, y_kmeans_gru, digits=3, target_names=target_names))

In [ ]:
svns_neu_gru = []
for i in range(0,len(Y_val)):
    neu = cosine(cl_32[i], centers_gru[2])/2
    svns_neu_gru.append(1-neu)
print(len(svns_neu_gru))

In [ ]:
svns_pos_gru = []
for i in range(0,len(Y_val)):
    pos = cosine(cl_32[i], centers_gru[0])/2
    svns_pos_gru.append(1-pos)
print(len(svns_pos_gru))

In [ ]:
svns_neg_gru = []
for i in range(0,len(Y_val)):
    neg = cosine(cl_32[i], centers_gru[1])/2
    svns_neg_gru.append(1-neg)
print(len(svns_neg_gru))

In [ ]:
pred_colour = []
for i in range(0,len(pred)):
    if y_kmeans_gru[i] == 0:
      pred_colour.append("Neutral")
    if y_kmeans_gru[i] == 1:
      pred_colour.append("Positive")
    if y_kmeans_gru[i] == 2:
      pred_colour.append("Negative")

test_df = pd.DataFrame({'SVNS Positive':svns_pos_gru, 'SVNS Negative':svns_neg_gru, 'SVNS Neutral':svns_neu_gru, 'Labels:':pred_colour})

fig = px.scatter_3d(test_df, x='SVNS Positive', y='SVNS Negative', z='SVNS Neutral', color='Labels:')
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale' : 'viridis',
    }
)
fig.update_layout(legend= {'itemsizing': 'constant'})
fig.update_layout(width = 850, height = 750)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))